In [1]:
import pandas as pd
import soynlp
from soynlp.noun import LRNounExtractor_v2
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
def clean_sents_df(company):
    try:
        target = pd.read_csv(f'./src/{company}_2year.csv',encoding='utf8').drop('Unnamed: 0',axis=1)
    except:
        target = pd.read_csv(f'./src/{company}_2year.csv',encoding='utf8')

    if company == 'sampro':
        target.rename(columns={'comment':'댓글'},inplace=True)
    df = target
    df['정제된 댓글'] = df['댓글'].str.replace('\\[삭제된 게시물의 답글\\]',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
    df = df[df['정제된 댓글'] != '']
    df = df.reset_index(drop=True)
    return  df

def corpus_save(company):
    df = clean_sents_df(company)
    df['정제된 댓글 길이'] = [len(str(i)) for i in df['정제된 댓글']]
    df = df[df['정제된 댓글 길이'] > 5]

    tp = [str(i) for i in list(df['정제된 댓글'])]
    save = '\n'.join(tp)
    f = open("./src/corpus_target.txt", 'a',encoding='utf8')
    f.write(save)
    f.close()

    
def corpus_init():
    ktop30_company = pd.read_excel('./src/KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
    company_set = list(ktop30_company)
    company_set.append('sampro')
    f = open("./src/corpus_target.txt", 'w',encoding='utf8')
    f.write('')
    f.close()
    for company in company_set:
        corpus_save(company)

def return_tokenizer():
    corpus = DoublespaceLineCorpus("./src/corpus_target.txt",iter_sent=True)
    noun_extractor = LRNounExtractor_v2(verbose=True)
    nouns = noun_extractor.train_extract(corpus)
    scores = {word:score.score for word, score in nouns.items()}
    tokenizer = LTokenizer(scores=scores)
    return tokenizer

In [3]:
corpus_init()
tokenizer = return_tokenizer()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1249734 from 2753662 sents. mem=0.370 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=9746776, mem=2.907 Gb
[Noun Extractor] batch prediction was completed for 289636 words
[Noun Extractor] checked compounds. discovered 280851 compounds
[Noun Extractor] postprocessing detaching_features : 249805 -> 199708
[Noun Extractor] postprocessing ignore_features : 199708 -> 199316
[Noun Extractor] postprocessing ignore_NJ : 199316 -> 196316
[Noun Extractor] 196316 nouns (280851 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=3.314 Gb                    
[Noun Extractor] 71.68 % eojeols are covered


In [4]:
target_df = clean_sents_df('삼성전자')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
target_df

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글
0,2022-07-11 23:42,요즘 한전 게시판 조용하네!ㅋ,2,0,0,요즘 한전 게시판 조용하네,"[요즘, 한전, 게시판, 조용, 하네]"
1,2022-07-11 23:42,1% 올려야 함,4,0,0,올려야 함,"[올려야, 함]"
2,2022-07-11 23:42,=자신들잘못된죄들=모르는=내로남불들=척결...,3,0,0,자신들잘못된죄들 모르는 내로남불들 척결,"[자신들, 잘못된죄들, 모르는, 내로남불, 들, 척결]"
3,2022-07-11 23:42,난 내일 가요,9,0,0,난 내일 가요,"[난, 내일, 가요]"
4,2022-07-11 23:42,실적으로 이만한 실적내는 기업 있으면 나...,6,0,0,실적으로 이만한 실적내는 기업 있으면 나,"[실적, 으로, 이만, 한, 실적, 내는, 기업, 있으면, 나]"
...,...,...,...,...,...,...,...
952447,2020-06-01 00:09,내8자,707,1,3,내 자,"[내, 자]"
952448,2020-06-01 00:08,하락낙폭저지 의 길,682,2,0,하락낙폭저지 의 길,"[하락낙폭저지, 의, 길]"
952449,2020-06-01 00:07,주주님들 오랜만에 오르니까 좋네요 ㅎㅎㅎ,697,1,0,주주님들 오랜만에 오르니까 좋네요,"[주주님, 들, 오랜만, 에, 오르니까, 좋네요]"
952450,2020-06-01 00:03,코덱스200선물인버스 곱배기..,725,0,0,코덱스 선물인버스 곱배기,"[코덱스, 선물, 인버스, 곱배기]"
